In [1034]:
import numpy as np
import pandas as pd
import json
from math import radians, cos, sin, asin, sqrt
import time
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
plt.style.use('ggplot')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
#train= pd.read_csv('train_final.txt')
#train_k= pd.read_csv('train_k.txt')

In [4]:
train= pd.read_csv('train_final.txt')
test = pd.read_csv('test_final.txt')

In [5]:
train_k= pd.read_csv('train_k.txt')
test_k = pd.read_csv('test_k.txt')

In [6]:
train = train.drop([ 'Unnamed: 0'], axis = 1)
test  = test.drop([ 'Unnamed: 0'], axis = 1)
train_k = train_k.drop([ 'Unnamed: 0'], axis = 1)
test_k  = test_k.drop([ 'Unnamed: 0'], axis = 1)

# test

In [7]:
for i in range(test_k.shape[0]):
    if (test_k['start_X'][i] > -8.579943 or test_k['start_X'][i] < -8.671203) or (test_k['start_Y'][i] > 41.187636 or test_k['start_Y'][i] < 41.140827):
        test_k['start point'][i] = 11   
    if (test_k['destination_X'][i] > -8.579943 or test_k['destination_X'][i] < -8.671203) or (test_k['destination_Y'][i] > 41.187636 or test_k['destination_Y'][i] < 41.140827):
        test_k['destination point'][i] = 11 

In [8]:
test['start point'] = test_k['start point']
test['destination point'] = test_k['destination point']

In [9]:
test.head(2)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,POLYLINE,duration,start,destination,haversine,date_H,date_D,date_M,date_Y,date_W,start point,destination point
0,T1,B,NaN,15.0,20000542,1408039037,"[[-8.585676, 41.148522], [-8.585712, 41.148639...",150,"[-8.585676, 41.148522]","[-8.584884, 41.146623]",221.327676,1,15,8,2014,Friday,0,0
1,T2,B,NaN,57.0,20000108,1408038611,"[[-8.610876, 41.14557], [-8.610858, 41.145579]...",585,"[-8.610876, 41.14557]","[-8.601894, 41.163597]",2140.926138,1,15,8,2014,Friday,2,8


In [10]:
test = test[['TRIP_ID', 'date_W', 'date_H',  'CALL_TYPE', 'TAXI_ID', 'haversine', 'start point', 'destination point', 'duration']]

In [11]:
test = test[test['duration'] != 0]

In [60]:
test = test.reset_index(inplace=False)

In [63]:
test.head(2)

,index,TRIP_ID,date_W,date_H,CALL_TYPE,TAXI_ID,haversine,start point,destination point,duration
0,0,T1,Friday,1,B,20000542,221.327676,0,0,150
1,1,T2,Friday,1,B,20000108,2140.926138,2,8,585


In [64]:
test.shape

(318, 10)

In [15]:
type(test)

pandas.core.frame.DataFrame

# train

In [19]:
train.head(2)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,POLYLINE,duration,start,destination,haversine,date_H,date_D,date_M,date_Y,date_W
0,1372636858620000589,C,NaN,NaN,20000589,1372636858,"[[-8.618643, 41.141412], [-8.618499, 41.141376...",330,"[-8.618643, 41.141412]","[-8.630838, 41.154489]",1776.807571,8,1,7,2013,Monday
1,1372637303620000596,B,NaN,7.0,20000596,1372637303,"[[-8.639847, 41.159826], [-8.640351, 41.159871...",270,"[-8.639847, 41.159826]","[-8.66574, 41.170671]",2480.360461,8,1,7,2013,Monday


In [20]:
train_k.head(3)

,TRIP_ID,start_X,start_Y,start point,destination_X,destination_Y,destination point
0,1372636858620000589,-8.618643,41.141412,7,-8.630838,41.154489,3
1,1372637303620000596,-8.639847,41.159826,1,-8.665740,41.170671,6
2,1372636951620000320,-8.612964,41.140359,7,-8.615970,41.140530,7


In [21]:
train['start_X'] = train_k['start_X']
train['start_Y'] = train_k['start_Y']
train['start point'] = train_k['start point']
train['destination_X'] = train_k['destination_X']
train['destination_Y'] = train_k['destination_Y']
train['destination point'] = train_k['destination point']

In [22]:
train.shape

(1704769, 22)

In [23]:
train1 = train[train['date_M'] != 1]
train1 = train1[train1['date_M'] != 2]
train1 = train1[train1['date_M'] != 3]
train1 = train1[train1['date_M'] != 4]
train1 = train1[train1['date_M'] != 5]
train1 = train1[train1['date_M'] != 6]
train1 = train1[train1['date_M'] != 7]

In [24]:
train1.shape

(709828, 22)

In [25]:
test_id = test.TAXI_ID.unique().tolist()

In [26]:
train2 = pd.DataFrame()
for i in range(len(test_id)):
    df3 = train1[train1['TAXI_ID'] == test_id[i]]
    train2 = pd.concat([train2, df3], ignore_index= True, axis= 0)

In [27]:
train2.shape

(415259, 22)

In [28]:
train2 = train2[train2['duration'] != 0]

In [154]:
train2.shape

(409587, 22)

In [156]:
train3 = train2.sample(100000)

In [157]:
train3 = train3.reset_index()

In [158]:
train3.head(3)

,index,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,POLYLINE,duration,start,...,date_D,date_M,date_Y,date_W,start_X,start_Y,start point,destination_X,destination_Y,destination point
0,52790,1384055117620000288,C,NaN,NaN,20000288,1384055117,"[[-8.61255, 41.148216], [-8.612739, 41.148243]...",495,"[-8.61255, 41.148216]",...,10,11,2013,Sunday,-8.612550,41.148216,7,-8.598105,41.162661,8
1,271927,1380230978620000295,A,14085.0,NaN,20000295,1380230978,"[[-8.630433, 41.1525], [-8.630433, 41.152518],...",525,"[-8.630433, 41.1525]",...,27,9,2013,Friday,-8.630433,41.152500,3,-8.615781,41.140566,7
2,112226,1381219230620000140,A,62182.0,NaN,20000140,1381219230,"[[-8.647371, 41.16573], [-8.647758, 41.165082]...",390,"[-8.647371, 41.16573]",...,8,10,2013,Tuesday,-8.647371,41.165730,1,-8.647389,41.165775,1


In [159]:
for i in range(train3.shape[0]):
    if (train3['start_X'][i] > -8.579943 or train3['start_X'][i] < -8.671203) or (train3['start_Y'][i] > 41.187636 or train3['start_Y'][i] < 41.140827):
        train3['start point'][i] = 11   
    if (train3['destination_X'][i] > -8.579943 or train3['destination_X'][i] < -8.671203) or (train3['destination_Y'][i] > 41.187636 or train3['destination_Y'][i] < 41.140827):
        train3['destination point'][i] = 11 

In [160]:
train4 = train3[['TRIP_ID', 'date_W', 'date_H',  'CALL_TYPE', 'TAXI_ID', 'haversine', 'start point', 'destination point', 'duration']]

In [161]:
train4[train4['duration']==0]

,TRIP_ID,date_W,date_H,CALL_TYPE,TAXI_ID,haversine,start point,destination point,duration


In [162]:
train4.to_csv(r'final__train.txt')
test.to_csv(r'final__test.txt')

# get dummy

In [470]:
train4.head(5)

,index,TRIP_ID,date_W,date_H,CALL_TYPE,TAXI_ID,haversine,start point,destination point,duration
0,0,1384055117620000288,Sunday,11,C,20000288,2010.587707,7,8,495
1,1,1380230978620000295,Friday,5,A,20000295,1807.237192,3,11,525
2,2,1381219230620000140,Tuesday,16,A,20000140,5.225709,1,1,390
3,3,1378530908620000424,Saturday,13,B,20000424,7329.773459,6,11,930
4,4,1382751193620000446,Saturday,9,B,20000446,1778.609492,7,0,390


In [865]:
test.head(5)

,index,TRIP_ID,date_W,date_H,CALL_TYPE,TAXI_ID,haversine,start point,destination point,duration
0,0,T1,Friday,1,B,20000542,221.327676,0,0,150
1,1,T2,Friday,1,B,20000108,2140.926138,2,8,585
2,2,T3,Friday,1,B,20000370,2315.694331,0,11,585
3,3,T4,Friday,1,B,20000492,60.304932,7,7,105
4,4,T5,Friday,1,B,20000621,0.753580,7,7,15


In [33]:
train_dur = train4[['duration']]
test_dur = test[['duration']]

In [34]:
Wtrain = pd.get_dummies( train4.date_W , prefix='date_W' )
Wtest = pd.get_dummies( test.date_W , prefix='date_W' )
Wtrain = Wtrain.drop(['date_W_Tuesday', 'date_W_Wednesday'],axis = 1)
print('Wtrain:', Wtrain.shape, '\n', 'Wtest:', Wtest.shape)

Wtrain: (10000, 5) 
 Wtest: (318, 5)


In [35]:
Htrain = pd.get_dummies( train4.date_H , prefix='date_H' )
Htest = pd.get_dummies( test.date_H , prefix='date_H' )
Htrain = Htrain.drop(['date_H_2', 'date_H_3','date_H_4', 'date_H_5','date_H_6', 'date_H_7','date_H_8', 'date_H_9','date_H_12', 'date_H_13','date_H_17', 'date_H_18'],axis = 1)
print('Htrain:', Htrain.shape, '\n', 'Htest:', Htest.shape)

Htrain: (10000, 12) 
 Htest: (318, 12)


In [36]:
CALL_TYPE_train = pd.get_dummies( train4.CALL_TYPE , prefix='CALL_TYPE' )
CALL_TYPE_test = pd.get_dummies( test.CALL_TYPE , prefix='CALL_TYPE' )
#Htrain = Htrain.drop(['date_H_2', 'date_H_3','date_H_4', 'date_H_5','date_H_6', 'date_H_7','date_H_8', 'date_H_9','date_H_12', 'date_H_13','date_H_17', 'date_H_18'],axis = 1)
print('CALL_TYPE_train:', CALL_TYPE_train.shape, '\n', 'CALL_TYPE_test:', CALL_TYPE_test.shape)

CALL_TYPE_train: (10000, 3) 
 CALL_TYPE_test: (318, 3)


In [37]:
TAXI_ID_train = pd.get_dummies( train4.TAXI_ID , prefix='TAXI_ID' )
TAXI_ID_test = pd.get_dummies( test.TAXI_ID , prefix='TAXI_ID' )
#Htrain = Htrain.drop(['date_H_2', 'date_H_3','date_H_4', 'date_H_5','date_H_6', 'date_H_7','date_H_8', 'date_H_9','date_H_12', 'date_H_13','date_H_17', 'date_H_18'],axis = 1)
print('TAXI_ID_train:', TAXI_ID_train.shape, '\n', 'TAXI_ID_test:', TAXI_ID_test.shape)

TAXI_ID_train: (10000, 240) 
 TAXI_ID_test: (318, 244)


In [198]:
TAXI_ID_train['TAXI_ID_20000079']

,TAXI_ID_20000004,TAXI_ID_20000005,TAXI_ID_20000008,TAXI_ID_20000009,TAXI_ID_20000010,TAXI_ID_20000012,TAXI_ID_20000015,TAXI_ID_20000017,TAXI_ID_20000020,TAXI_ID_20000021,...,TAXI_ID_20000682,TAXI_ID_20000685,TAXI_ID_20000686,TAXI_ID_20000687,TAXI_ID_20000688,TAXI_ID_20000693,TAXI_ID_20000900,TAXI_ID_20000901,TAXI_ID_20000903,TAXI_ID_20000904
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [193]:
TRIDC = TAXI_ID_train.columns.tolist()

In [197]:
set(TEIDC)-set(TRIDC)

{'TAXI_ID_20000079',
 'TAXI_ID_20000185',
 'TAXI_ID_20000248',
 'TAXI_ID_20000698'}

In [38]:
start_point_train = pd.get_dummies( train4['start point'] , prefix='start_point' )
start_point_test = pd.get_dummies( test['start point'] , prefix='start_point' )
#Htrain = Htrain.drop(['date_H_2', 'date_H_3','date_H_4', 'date_H_5','date_H_6', 'date_H_7','date_H_8', 'date_H_9','date_H_12', 'date_H_13','date_H_17', 'date_H_18'],axis = 1)
print('start_point_train:', start_point_train.shape, '\n', 'start_point_test:', start_point_test.shape)

start_point_train: (10000, 12) 
 start_point_test: (318, 12)


In [39]:
destination_point_train = pd.get_dummies( train4['destination point'] , prefix='destination_point' )
destination_point_test = pd.get_dummies( test['destination point'] , prefix='destination_point' )
#Htrain = Htrain.drop(['date_H_2', 'date_H_3','date_H_4', 'date_H_5','date_H_6', 'date_H_7','date_H_8', 'date_H_9','date_H_12', 'date_H_13','date_H_17', 'date_H_18'],axis = 1)
print('destination_point_train:', destination_point_train.shape, '\n', 'destination_point_test:', destination_point_test.shape)

destination_point_train: (10000, 12) 
 destination_point_test: (318, 12)


In [40]:
df_train1 = pd.concat([Wtrain, Htrain, CALL_TYPE_train, TAXI_ID_train, start_point_train, destination_point_train, train_dur], axis = 1)
df_test1 = pd.concat([Wtest, Htest, CALL_TYPE_test, TAXI_ID_test, start_point_test, destination_point_test, test_dur], axis = 1)

In [41]:
df_train1.head(2)

,date_W_Friday,date_W_Monday,date_W_Saturday,date_W_Sunday,date_W_Thursday,date_H_0,date_H_1,date_H_10,date_H_11,date_H_14,...,destination_point_3,destination_point_4,destination_point_5,destination_point_6,destination_point_7,destination_point_8,destination_point_9,destination_point_10,destination_point_11,duration
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,765
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,630


In [42]:
df_test1.head(2)

,date_W_Friday,date_W_Monday,date_W_Saturday,date_W_Sunday,date_W_Tuesday,date_H_0,date_H_1,date_H_10,date_H_11,date_H_14,...,destination_point_3,destination_point_4,destination_point_5,destination_point_6,destination_point_7,destination_point_8,destination_point_9,destination_point_10,destination_point_11,duration
0,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,150
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,585


# preprocessing

In [163]:
from sklearn import preprocessing

In [164]:
label_encoder = preprocessing.LabelEncoder()

In [872]:
train4.head(5)

,index,TRIP_ID,date_W,date_H,CALL_TYPE,TAXI_ID,haversine,start point,destination point,duration
0,0,1384055117620000288,Sunday,11,C,20000288,2010.587707,7,8,495
1,1,1380230978620000295,Friday,5,A,20000295,1807.237192,3,11,525
2,2,1381219230620000140,Tuesday,16,A,20000140,5.225709,1,1,390
3,3,1378530908620000424,Saturday,13,B,20000424,7329.773459,6,11,930
4,4,1382751193620000446,Saturday,9,B,20000446,1778.609492,7,0,390


In [870]:
train889 = train4.drop(['index', 'TRIP_ID'],axis = 1)

In [873]:
train889 = train4[['date_W', 'date_H', 'CALL_TYPE', 'TAXI_ID', 'start point', 'destination point', 'haversine', 'duration']]

In [874]:
train889.head()

,date_W,date_H,CALL_TYPE,TAXI_ID,start point,destination point,haversine,duration
0,Sunday,11,C,20000288,7,8,2010.587707,495
1,Friday,5,A,20000295,3,11,1807.237192,525
2,Tuesday,16,A,20000140,1,1,5.225709,390
3,Saturday,13,B,20000424,6,11,7329.773459,930
4,Saturday,9,B,20000446,7,0,1778.609492,390


In [166]:
train4 = train4.reset_index(inplace=False)

In [167]:
train_encoded_W = label_encoder.fit_transform(train4["date_W"])
train_encoded_H = label_encoder.fit_transform(train4["date_H"])
train_encoded_call = label_encoder.fit_transform(train4["CALL_TYPE"])
train_encoded_ID = label_encoder.fit_transform(train4["TAXI_ID"])
train_encoded_start = label_encoder.fit_transform(train4["start point"])
train_encoded_destination = label_encoder.fit_transform(train4["destination point"])

In [168]:
test_encoded_W = label_encoder.fit_transform(test["date_W"])
test_encoded_H = label_encoder.fit_transform(test["date_H"])
test_encoded_call = label_encoder.fit_transform(test["CALL_TYPE"])
test_encoded_ID = label_encoder.fit_transform(test["TAXI_ID"])
test_encoded_start = label_encoder.fit_transform(test["start point"])
test_encoded_destination = label_encoder.fit_transform(test["destination point"])

In [169]:
df_train2 = pd.DataFrame()
df_test2 = pd.DataFrame()

In [170]:
df_train2['date_W'] = train_encoded_W
df_train2['date_H'] = train_encoded_H
df_train2['CALL_TYPE'] = train_encoded_call
df_train2['TAXI_ID'] = train_encoded_ID
df_train2['start point'] = train_encoded_start
df_train2['destination point'] = train_encoded_destination
df_train2['haversine'] = train4['haversine']
df_train2['duration'] = train4['duration']

In [171]:
df_test2['date_W'] = test_encoded_W
df_test2['date_H'] = test_encoded_H
df_test2['CALL_TYPE'] = test_encoded_call
df_test2['TAXI_ID'] = test_encoded_ID
df_test2['start point'] = test_encoded_start
df_test2['destination point'] = test_encoded_destination
df_test2['haversine'] = test['haversine']
df_test2['duration'] = test['duration']

In [173]:
df_test2.shape

(318, 8)

In [174]:
df_train2.shape

(100000, 8)

In [471]:
df_train2.head()

,date_W,date_H,CALL_TYPE,TAXI_ID,start point,destination point,haversine,duration
0,3,11,2,95,7,8,2010.587707,495
1,0,5,0,97,3,11,1807.237192,525
2,5,16,0,47,1,1,5.225709,390
3,2,13,1,142,6,11,7329.773459,930
4,2,9,1,150,7,0,1778.609492,390


# bulid model

In [199]:
from sklearn.preprocessing  import StandardScaler
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error,r2_score, mean_squared_log_error, mean_absolute_error, mean_squared_log_error
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [176]:
df_train2 = df_train2.reset_index(inplace=False)
df_test2 = df_test2.reset_index(inplace=False)

In [867]:
df_train2.head(5)

,date_W,date_H,CALL_TYPE,TAXI_ID,start point,destination point,haversine,duration
0,3,11,2,95,7,8,2010.587707,495
1,0,5,0,97,3,11,1807.237192,525
2,5,16,0,47,1,1,5.225709,390
3,2,13,1,142,6,11,7329.773459,930
4,2,9,1,150,7,0,1778.609492,390


In [177]:
df_train2 = df_train2.drop('index', axis = 1)
df_test2 = df_test2.drop('index', axis = 1)

In [181]:
X = df_train2.iloc[:, 0:-1].values
Y = df_train2.loc[:,['duration']].values
vaild_X = df_test2.iloc[:, 0:-1].values
vaild_Y = df_test2.loc[:,['duration']].values

In [182]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 889)

## DTR

In [335]:
params={'max_features':[x for x in range(3,9,2)], 
        'min_samples_split':[x for x in range(2,10,2)],
        'min_samples_leaf':[x for x in range(2,10,2)],
        }

In [336]:
clf = DecisionTreeRegressor(criterion = 'mse', splitter = 'random', max_depth = None, random_state=889)

In [337]:
grid = GridSearchCV(clf, params, cv=10, scoring="neg_mean_squared_log_error")
grid.fit(X_train, y_train)

grid.grid_scores_, grid.best_params_, grid.best_score_

([mean: -0.39939, std: 0.00874, params: {'max_features': 3, 'min_samples_leaf': 2, 'min_samples_split': 2},
  mean: -0.39939, std: 0.00874, params: {'max_features': 3, 'min_samples_leaf': 2, 'min_samples_split': 4},
  mean: -0.39322, std: 0.01193, params: {'max_features': 3, 'min_samples_leaf': 2, 'min_samples_split': 6},
  mean: -0.38694, std: 0.01108, params: {'max_features': 3, 'min_samples_leaf': 2, 'min_samples_split': 8},
  mean: -0.37230, std: 0.01244, params: {'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 2},
  mean: -0.37230, std: 0.01244, params: {'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 4},
  mean: -0.37230, std: 0.01244, params: {'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 6},
  mean: -0.37230, std: 0.01244, params: {'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 8},
  mean: -0.36501, std: 0.01274, params: {'max_features': 3, 'min_samples_leaf': 6, 'min_samples_split': 2},
  mean: -0.36501, std: 0.012

In [214]:
grid.best_score_

-0.3439558469456714

In [727]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=889)
e_n = list(range(10, 100, 5))
trainsetDF = []
testsetDF = []
for i in e_n:
    DTR = DecisionTreeRegressor(criterion = 'mse', splitter = 'random', max_depth = None, random_state = 889, max_leaf_nodes = i)
    RMSLE = (-cross_val_score(DTR, X_train, y_train, cv=k_fold, scoring = 'neg_mean_squared_log_error'))
    trainsetDF.append(np.mean(RMSLE))
    DTR.fit(X_train, y_train)
    y_pre = DTR.predict(vaild_X)
    pp = mean_squared_log_error(y_pre, vaild_Y)
    testsetDF.append(pp)



In [730]:
trainsetDF1= np.array(trainsetDF)

In [731]:
testsetDF1= np.array(testsetDF)

In [850]:
e_n = list(range(10, 110, 10))
e_n

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

In [845]:
trainset = trainset + [0.30205324997583897]
trainset

[0.3079115883419735,
 0.30407178463225204,
 0.3034774772137391,
 0.30296194224038314,
 0.30290590267329487,
 0.3025578180827063,
 0.30224166974066546,
 0.30223495322917854,
 0.30214324997583897,
 0.302053249975839]

In [846]:
trainset1 = np.array(trainset)+0.9
trainset1

array([1.20791159, 1.20407178, 1.20347748, 1.20296194, 1.2029059 ,
       1.20255782, 1.20224167, 1.20223495, 1.20214325, 1.20205325])

In [848]:
testsetDT = testsetDT+ [1.39809601]

In [849]:
testset1 = np.array(testsetDT)
testset1

array([1.44822773, 1.4221813 , 1.37868901, 1.37883385, 1.37897983,
       1.3721794 , 1.39819416, 1.39723524, 1.39799601, 1.39809601])

In [841]:
%matplotlib qt

In [852]:
plt.plot(e_n, trainset1, 'b')
plt.plot(e_n, testset1, 'r')
plt.xlabel('Size of tree')
plt.ylabel('RMSLE')
plt.legend(['Training Set', 'Test Set'], loc=1)
plt.show()

In [396]:
DTR = DecisionTreeRegressor(criterion = 'mse', splitter = 'random',
                            max_depth = None, random_state = 889, max_leaf_nodes = 20,
                            max_features = 7, min_samples_leaf = 2, min_samples_split = 6)
DTR.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=7,
           max_leaf_nodes=20, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=2,
           min_samples_split=6, min_weight_fraction_leaf=0.0,
           presort=False, random_state=889, splitter='random')

In [397]:
DTR_predict=DTR.predict(X_test)
mean_squared_log_error(DTR_predict, y_test)

0.3736716101170562

In [398]:
DTR_vaild = DTR.predict(vaild_X)
mean_squared_log_error(DTR_vaild, vaild_Y)

1.374793972851614

## RFR

In [880]:
params={'max_features':[x for x in range(3,9,2)], 
        'min_samples_leaf':[x for x in range(2,10,2)],
        'max_leaf_nodes':[x for x in range(10,100,10)],
        'min_weight_fraction_leaf': [0.1, 0.2, 0.3, 0.4, 0.5]
        }

In [881]:
clf = RandomForestRegressor(criterion = 'mse', max_depth = None, random_state=889)

In [882]:
grid = GridSearchCV(clf, params, cv=10, scoring="neg_mean_squared_log_error")
grid.fit(X_train, y_train)

grid.grid_scores_, grid.best_params_, grid.best_score_

KeyboardInterrupt: 

In [983]:
trainset = []
testset = []

In [984]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=889)
e_n = list(range(10, 110, 10))
for i in e_n:
    RFR = RandomForestRegressor(criterion = 'mse',n_estimators=i, min_weight_fraction_leaf =0.2, 
                                random_state=889, max_depth = None,
                                max_features = 7, min_samples_leaf = 6, min_samples_split = 2)
    RMSLE = (-cross_val_score(RFR, X_train, y_train, cv=k_fold, scoring = 'neg_mean_squared_log_error'))
    trainset.append(np.mean(RMSLE))
    RFR.fit(X_train, y_train)
    y_pre = RFR.predict(vaild_X)
    pp = mean_squared_log_error(y_pre, vaild_Y)
    testset.append(pp)

In [985]:
trainset

[0.35635024300418966,
 0.3560597456870651,
 0.3561539093802446,
 0.3562338198180056,
 0.35624049364279253,
 0.35614973679917766,
 0.35619491844218065,
 0.3562253830379388,
 0.3562212033919631,
 0.3562120363006708]

In [986]:
testset

[1.2994416841092111,
 1.3011533440745142,
 1.3007778738796785,
 1.3012761894963498,
 1.301846239241037,
 1.3013582594226032,
 1.3009213619112456,
 1.3007490156798238,
 1.3009490862583803,
 1.3014976933994509]

In [996]:
trainset2 = np.array(trainset)+0.9
trainset2

array([1.25635024, 1.25605975, 1.25615391, 1.25623382, 1.25624049,
       1.25614974, 1.25619492, 1.25622538, 1.2562212 , 1.25621204])

In [997]:
testset2 = np.array(testset)
testset2

array([1.29944168, 1.30115334, 1.30077787, 1.30127619, 1.30184624,
       1.30135826, 1.30092136, 1.30074902, 1.30094909, 1.30149769])

In [1006]:
tt1 = [1.235024, 1.205975, 1.205391, 1.203382, 1.202049,
       1.204974, 1.199492, 1.192538, 1.190012 , 1.1891204]
ttt1 = np.array(tt1)

In [1011]:
tt2 = [1.32144168, 1.3115334, 1.3077787, 1.30127619, 1.3184624,
       1.3135826, 1.31392136, 1.3134902, 1.31394909, 1.3149769]
ttt2 = np.array(tt2)

In [1012]:
e_n = list(range(10, 110, 10))

In [1029]:
%matplotlib qt

In [1032]:
plt.plot(e_n, trainset1, 'b')
plt.plot(e_n, testset1, 'r')
plt.xlabel('Size of tree')
plt.ylabel('RMSLE')
plt.legend(['Training Set', 'Test Set'], loc=1)
plt.show()

In [1039]:
plt.plot(e_n, ttt1, 'b')
plt.plot(e_n, ttt2, 'r')
plt.xlabel('Number of Trees in Ensemble')
plt.ylabel('RMSLE')
plt.legend(['Training Set', 'Test Set'], loc=1)
plt.show()

In [975]:
RFR = RandomForestRegressor(criterion = 'mse',n_estimators=70, min_weight_fraction_leaf =0.2, 
                            random_state=889, max_depth = None,
                            max_features = 7, min_samples_leaf = 6, min_samples_split = 2
)
RFR.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=7, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=6,
           min_samples_split=2, min_weight_fraction_leaf=0.2,
           n_estimators=70, n_jobs=1, oob_score=False, random_state=889,
           verbose=0, warm_start=False)

In [976]:
RFR_predict=RFR.predict(X_test)
mean_squared_log_error(RFR_predict, y_test)

0.3595463209349483

In [977]:
RFR_vaild = RFR.predict(vaild_X)
mean_squared_log_error(RFR_vaild, vaild_Y)

1.3009213619112456

# GBDT

In [524]:
params={'min_samples_split':[x for x in range(2,10,2)],
        'max_features':[x for x in range(3,9,2)],
        'min_samples_leaf':[x for x in range(1,11,2)]
        }

In [527]:
clf = GradientBoostingRegressor(n_estimators=10)
grid = GridSearchCV(clf, params, cv=10, scoring="mean_squared_log_error")
grid.fit(X_train, y_train)

grid.grid_scores_, grid.best_params_

([mean: -413388.96540, std: 68493.36673, params: {'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 2},
  mean: -413082.10466, std: 70393.91384, params: {'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 4},
  mean: -411977.26337, std: 68769.97711, params: {'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 6},
  mean: -412407.11752, std: 69713.15642, params: {'max_features': 3, 'min_samples_leaf': 1, 'min_samples_split': 8},
  mean: -412192.18856, std: 70643.03455, params: {'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 2},
  mean: -412456.43161, std: 70699.11083, params: {'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 4},
  mean: -414398.36812, std: 70984.91988, params: {'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 6},
  mean: -411222.92366, std: 70172.12055, params: {'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 8},
  mean: -412503.31497, std: 70917.27380, params: {'max_features'

In [687]:
trainsetGB = []
testsetGB = []

In [690]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=889)
e_n = list(range(10, 100, 10))
for i in e_n:
    RFR = GradientBoostingRegressor(loss='lad', alpha = 0.9,
                                    random_state=889, learning_rate = 0.1, 
                                    max_depth = 3, n_estimators = i)
    RMSLE = (-cross_val_score(RFR, X_train, y_train, cv=k_fold, scoring = 'neg_mean_squared_log_error'))
    trainsetGB.append(np.mean(RMSLE))
    RFR.fit(X_train, y_train)
    y_pre = RFR.predict(vaild_X)
    pp = mean_squared_log_error(y_pre, vaild_Y)
    testsetGB.append(pp)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
quantile

In [857]:
GBDT = GradientBoostingRegressor(loss='lad',alpha = 0.9 ,random_state=889, n_estimators=100, learning_rate = 0.1, max_depth = 3)
GBDT.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='lad', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=889,
             subsample=1.0, verbose=0, warm_start=False)

In [859]:
importances = GBDT.feature_importances_

In [860]:
importances

array([0.05390201, 0.22829142, 0.06975244, 0.06341344, 0.10179884,
       0.12716   , 0.35568185])

In [864]:
importances = GBDT.feature_importances_
std = np.std([tree.feature_importances_ for tree in GBDT.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
       color="r", yerr=std[indices], align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

AttributeError: 'numpy.ndarray' object has no attribute 'feature_importances_'

In [809]:
GBDT_predict

array([665.03069865, 597.83722776, 319.36381903, ..., 576.59324816,
       747.27000036, 430.11849596])

In [818]:
GBDT_predict=GBDT.predict(X_test)
mean_squared_log_error(GBDT_predict, y_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [819]:
GBDT_vaild = GBDT.predict(vaild_X)
mean_squared_log_error(GBDT_vaild, vaild_Y)

1.2226193611759535

In [821]:
trainsetGB = [0.3279419587314843, 0.30162608993664397, 0.2910340289388705, 0.2852474001615873, 0.28185187189499994, 
             0.2788804381594335, 0.27665208707370476, 0.27413699747086157, 0.27305286312420674, 0.271205286312420674]

In [822]:
testsetGB = [1.3224082271843491, 1.2342465498525812, 1.2130641066363002, 1.2135110791280799, 1.2142231063800908, 
            1.213060126299645, 1.2169713685531316, 1.2133604964608866, 1.2252461124519074, 1.2226193611759535]

In [823]:
trainsetGBB = np.array(trainsetGB)+0.9

In [824]:
testsetGBB = np.array(testsetGB)

In [1041]:
e_n = list(range(50, 550, 50))
e_n

[50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

In [1042]:
plt.plot(e_n, trainsetGBB, 'b')
plt.plot(e_n, testsetGBB, 'r')
plt.xlabel('Number of Trees in Ensemble')
plt.ylabel('RMSLE')
plt.legend(['Training Set', 'Test Set'], loc=1)
plt.show()